# Package Load

In [1]:
import preprocessing
import tensorflow as tf
import numpy as np
import random
import sys
import utils
import pickle

# preprocessing : 이미지 파일로 존재하는 데이터를 numpy와 onehotencoding 라벨로 바꿔주는 모듈
# tensorflow : 텐서플로우 , 일반적으로 tf로 줄여서 사용
# numpy : fundamental package for scientific computing with Python
# pickle : python 의 데이터 형식을 그대로 저장하기 위한 파이썬 기본 패키지

# Image Loading

In [2]:
train_imgs , train_labs , test_imgs , test_labs=preprocessing.get_cifar_data()

def next_batch(imgs , labs , batch_size):
    indices=random.sample(range(len(imgs)) , batch_size)
    batch_xs=imgs[indices]
    batch_ys=labs[indices]
    return batch_xs , batch_ys

# preprocessing 모듈에서 train_imgs , train_labs , test_imgs , test_labs 를 불러 온다.
# fully connected 모델을 위해 32x32x3 크기의 이미지를 3072 으로 reshape 한다.
# next_batch 함수는 batch_size 수 만큼 random 으로 추출하여 train batch 를 생성하는 함수

# Set Placeholder And Variables

In [3]:
height = 32
width = 32
color_ch=3
n_classes = 10
learning_rate=0.001
max_iter=10000
check_point=100
x_ =tf.placeholder(tf.float32, [ None , height , width , color_ch ])
y_ =tf.placeholder( tf.int32 , [ None , n_classes ])

# Placeholder 는 차후에 입력할 값
# Variables 는 weight 로 학습시 변하는 값

# Network Modeling

In [4]:
out_ch=28
w1=tf.get_variable("w1" , [7,7,color_ch , out_ch] , initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.constant(0.1) ,out_ch)
s1=[1,1,1,1]
p1='SAME'
layer1=tf.nn.conv2d(x_ , w1 , s1 , p1 )+b1
layer1=tf.nn.relu(layer1)


out_ch2=64
w2=tf.get_variable("w2" , [5,5,out_ch, out_ch2] , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.constant(0.1) ,out_ch2)
s2=[1,1,1,1]
layer2=tf.nn.conv2d(layer1, w2 , s2, padding='SAME')+b2
layer2=tf.nn.relu(layer2)


pool_s = [1,2,2,1]
layer3 = tf.nn.max_pool(layer2, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')

out_ch3=128
w3=tf.get_variable("w3" , [5,5,out_ch2, out_ch3] , initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.constant(0.1) ,out_ch3)
s3=[1,1,1,1]
layer4=tf.nn.conv2d(layer3, w3 , s3, padding='SAME')+b3
layer4=tf.nn.relu(layer4)

out_ch4=128
w4=tf.get_variable("w4" , [5,5,out_ch3, out_ch4] , initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.constant(0.1) ,out_ch4)
s4=[1,1,1,1]
layer4=tf.nn.conv2d(layer4, w4 , s4, padding='SAME')+b4
layer4=tf.nn.relu(layer4)

layer5 = tf.nn.max_pool(layer4, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')

#fully connected layer = affine layer

end_conv_layer=layer5
flatten_layer=tf.contrib.layers.flatten(end_conv_layer)
length=flatten_layer.get_shape()[1]
fc_w1=tf.get_variable("fc_w1" ,[length,n_classes])
fc_b1=tf.Variable(tf.constant(0.1) , n_classes)
y_conv=tf.matmul(flatten_layer ,fc_w1 )+fc_b1

# 4개의 convolution neural network 와 1개의 fully connected_layer 로 구성
# 2개의 convolution layer 를 거친 후 각 각 max pooling 적용 / max pooling 후에는 activation map 의 가로 세로 크기가 절반이 된다.
# stride : 좌우로 몇 칸 씩 커널을 이동 할 것인지에 대한 값
# padding : convolution 전후로 activation map 의 크기를 조정하기 위한 값 , SAME 을 입력하면 항상 convolution 전후의 크기가 같다.
# weight : [커널의 가로크기, 커널의 세로크기, input activation map 의 채널 크기, output activiation map 의 채널 크기]

# Define Functions

In [5]:
pred=tf.nn.softmax(y_conv)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred , labels=y_) , name='cost')
train=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost,name='train')
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32), name='accuracy')

# softmax : 결과값을 각 class 에 해당하는 확률값을로 리턴
# softmax_cross_entropy_with_logits : 실제 정답인 onehot vector 와 예측값 pred 를 차이를 cross_entropy 로 계산
# tf.train.GradientDescentOptimizer : cost 가 최소와 되도록 weight를 조정하는 함수
# accuracy : 실제 값과 예측값의 일치률

# Training

In [6]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

batch_iteration = 100    
training_epochs = 2000

train_cost_list = []
test_cost_list = []
train_acc_list = []
test_acc_list = []

# Tensorflow 학습을 위한 session 정의
# saver : 학습한 weight 와 graph 를 저장

# sess.run 을 통해 함수를 실행 , feed_dict 는 placeholder 에 들어갈 값


for epoch in range(training_epochs):
    
    train_avg_cost = 0.
    train_avg_acc = 0.
    test_avg_cost = 0.
    test_avg_acc = 0.
    
    for batch in range(batch_iteration) :
        batch_xs , batch_ys = next_batch(train_imgs , train_labs , 60)
        
        sess.run(train, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_cost += sess.run(cost, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_acc += sess.run(accuracy, feed_dict={x_:batch_xs , y_ : batch_ys})
        
    train_avg_cost = train_avg_cost / batch_iteration
    train_avg_acc = train_avg_acc / batch_iteration
    
    test_avg_cost = sess.run(cost, feed_dict={x_: test_imgs, y_: test_labs})
    test_avg_acc = sess.run(accuracy, feed_dict={x_: test_imgs, y_: test_labs})
    
    
    print "##################################"
    print ("Epoch: %03d/%03d train cost: %.4f" % (epoch, training_epochs, train_avg_cost))
    print ("Epoch: %03d/%03d train acc: %.4f" % (epoch, training_epochs, train_avg_acc))
    print ("Epoch: %03d/%03d test cost: %.4f" % (epoch, training_epochs, test_avg_cost))
    print ("Epoch: %03d/%03d test acc: %.4f" % (epoch, training_epochs, test_avg_acc))    
    
    train_cost_list.append(train_avg_cost)
    test_cost_list.append(test_avg_cost)
    train_acc_list.append(train_avg_acc)
    test_acc_list.append(test_avg_acc)
   
    
    
saver.save(sess, "model/cifar_deep_convolution.ckpt")

with open('cost_acc/cifar_deep_convolution_train_cost', 'wb') as fp:
    pickle.dump(train_cost_list, fp)
with open('cost_acc/cifar_deep_convolution_test_cost', 'wb') as fp:
    pickle.dump(test_cost_list, fp)
with open('cost_acc/cifar_deep_convolution_train_acc', 'wb') as fp:
    pickle.dump(train_acc_list, fp)
with open('cost_acc/cifar_deep_convolution_test_acc', 'wb') as fp:
    pickle.dump(test_acc_list, fp)

##################################
Epoch: 000/2000 train cost: 2.3026
Epoch: 000/2000 train acc: 0.1102
Epoch: 000/2000 test cost: 2.3025
Epoch: 000/2000 test acc: 0.1035
##################################
Epoch: 001/2000 train cost: 2.3028
Epoch: 001/2000 train acc: 0.0915
Epoch: 001/2000 test cost: 2.3024
Epoch: 001/2000 test acc: 0.1020
##################################
Epoch: 002/2000 train cost: 2.3023
Epoch: 002/2000 train acc: 0.1033
Epoch: 002/2000 test cost: 2.3023
Epoch: 002/2000 test acc: 0.1025
##################################
Epoch: 003/2000 train cost: 2.3022
Epoch: 003/2000 train acc: 0.1053
Epoch: 003/2000 test cost: 2.3022
Epoch: 003/2000 test acc: 0.1050
##################################
Epoch: 004/2000 train cost: 2.3021
Epoch: 004/2000 train acc: 0.1033
Epoch: 004/2000 test cost: 2.3021
Epoch: 004/2000 test acc: 0.1050
##################################
Epoch: 005/2000 train cost: 2.3020
Epoch: 005/2000 train acc: 0.1050
Epoch: 005/2000 test cost: 2.3020
Epoch: 

##################################
Epoch: 048/2000 train cost: 2.2921
Epoch: 048/2000 train acc: 0.1317
Epoch: 048/2000 test cost: 2.2923
Epoch: 048/2000 test acc: 0.1244
##################################
Epoch: 049/2000 train cost: 2.2902
Epoch: 049/2000 train acc: 0.1318
Epoch: 049/2000 test cost: 2.2914
Epoch: 049/2000 test acc: 0.1218
##################################
Epoch: 050/2000 train cost: 2.2905
Epoch: 050/2000 train acc: 0.1275
Epoch: 050/2000 test cost: 2.2905
Epoch: 050/2000 test acc: 0.1334
##################################
Epoch: 051/2000 train cost: 2.2903
Epoch: 051/2000 train acc: 0.1335
Epoch: 051/2000 test cost: 2.2896
Epoch: 051/2000 test acc: 0.1372
##################################
Epoch: 052/2000 train cost: 2.2885
Epoch: 052/2000 train acc: 0.1447
Epoch: 052/2000 test cost: 2.2886
Epoch: 052/2000 test acc: 0.1517
##################################
Epoch: 053/2000 train cost: 2.2877
Epoch: 053/2000 train acc: 0.1560
Epoch: 053/2000 test cost: 2.2874
Epoch: 

##################################
Epoch: 096/2000 train cost: 2.1927
Epoch: 096/2000 train acc: 0.2708
Epoch: 096/2000 test cost: 2.1987
Epoch: 096/2000 test acc: 0.2649
##################################
Epoch: 097/2000 train cost: 2.1926
Epoch: 097/2000 train acc: 0.2718
Epoch: 097/2000 test cost: 2.1967
Epoch: 097/2000 test acc: 0.2650
##################################
Epoch: 098/2000 train cost: 2.1864
Epoch: 098/2000 train acc: 0.2740
Epoch: 098/2000 test cost: 2.1951
Epoch: 098/2000 test acc: 0.2658
##################################
Epoch: 099/2000 train cost: 2.1928
Epoch: 099/2000 train acc: 0.2695
Epoch: 099/2000 test cost: 2.1958
Epoch: 099/2000 test acc: 0.2656
##################################
Epoch: 100/2000 train cost: 2.1960
Epoch: 100/2000 train acc: 0.2682
Epoch: 100/2000 test cost: 2.1927
Epoch: 100/2000 test acc: 0.2685
##################################
Epoch: 101/2000 train cost: 2.1907
Epoch: 101/2000 train acc: 0.2675
Epoch: 101/2000 test cost: 2.1920
Epoch: 

##################################
Epoch: 144/2000 train cost: 2.1366
Epoch: 144/2000 train acc: 0.3318
Epoch: 144/2000 test cost: 2.1600
Epoch: 144/2000 test acc: 0.2957
##################################
Epoch: 145/2000 train cost: 2.1403
Epoch: 145/2000 train acc: 0.3223
Epoch: 145/2000 test cost: 2.1574
Epoch: 145/2000 test acc: 0.2991
##################################
Epoch: 146/2000 train cost: 2.1358
Epoch: 146/2000 train acc: 0.3290
Epoch: 146/2000 test cost: 2.1550
Epoch: 146/2000 test acc: 0.2997
##################################
Epoch: 147/2000 train cost: 2.1440
Epoch: 147/2000 train acc: 0.3150
Epoch: 147/2000 test cost: 2.1575
Epoch: 147/2000 test acc: 0.2970
##################################
Epoch: 148/2000 train cost: 2.1395
Epoch: 148/2000 train acc: 0.3252
Epoch: 148/2000 test cost: 2.1555
Epoch: 148/2000 test acc: 0.2980
##################################
Epoch: 149/2000 train cost: 2.1428
Epoch: 149/2000 train acc: 0.3188
Epoch: 149/2000 test cost: 2.1555
Epoch: 

##################################
Epoch: 192/2000 train cost: 2.1215
Epoch: 192/2000 train acc: 0.3418
Epoch: 192/2000 test cost: 2.1374
Epoch: 192/2000 test acc: 0.3173
##################################
Epoch: 193/2000 train cost: 2.1175
Epoch: 193/2000 train acc: 0.3453
Epoch: 193/2000 test cost: 2.1374
Epoch: 193/2000 test acc: 0.3175
##################################
Epoch: 194/2000 train cost: 2.1202
Epoch: 194/2000 train acc: 0.3448
Epoch: 194/2000 test cost: 2.1355
Epoch: 194/2000 test acc: 0.3186
##################################
Epoch: 195/2000 train cost: 2.1215
Epoch: 195/2000 train acc: 0.3447
Epoch: 195/2000 test cost: 2.1349
Epoch: 195/2000 test acc: 0.3216
##################################
Epoch: 196/2000 train cost: 2.1123
Epoch: 196/2000 train acc: 0.3508
Epoch: 196/2000 test cost: 2.1341
Epoch: 196/2000 test acc: 0.3202
##################################
Epoch: 197/2000 train cost: 2.1178
Epoch: 197/2000 train acc: 0.3477
Epoch: 197/2000 test cost: 2.1356
Epoch: 

...

##################################
Epoch: 1807/2000 train cost: 1.7560
Epoch: 1807/2000 train acc: 0.7122
Epoch: 1807/2000 test cost: 1.8935
Epoch: 1807/2000 test acc: 0.5659
##################################
Epoch: 1808/2000 train cost: 1.7395
Epoch: 1808/2000 train acc: 0.7278
Epoch: 1808/2000 test cost: 1.8864
Epoch: 1808/2000 test acc: 0.5708
##################################
Epoch: 1809/2000 train cost: 1.7471
Epoch: 1809/2000 train acc: 0.7218
Epoch: 1809/2000 test cost: 1.8867
Epoch: 1809/2000 test acc: 0.5736
##################################
Epoch: 1810/2000 train cost: 1.7576
Epoch: 1810/2000 train acc: 0.7097
Epoch: 1810/2000 test cost: 1.8885
Epoch: 1810/2000 test acc: 0.5705
##################################
Epoch: 1811/2000 train cost: 1.7470
Epoch: 1811/2000 train acc: 0.7210
Epoch: 1811/2000 test cost: 1.8913
Epoch: 1811/2000 test acc: 0.5668
##################################
Epoch: 1812/2000 train cost: 1.7535
Epoch: 1812/2000 train acc: 0.7162
Epoch: 1812/2000 te

##################################
Epoch: 1854/2000 train cost: 1.7353
Epoch: 1854/2000 train acc: 0.7323
Epoch: 1854/2000 test cost: 1.8826
Epoch: 1854/2000 test acc: 0.5770
##################################
Epoch: 1855/2000 train cost: 1.7390
Epoch: 1855/2000 train acc: 0.7280
Epoch: 1855/2000 test cost: 1.8852
Epoch: 1855/2000 test acc: 0.5736
##################################
Epoch: 1856/2000 train cost: 1.7457
Epoch: 1856/2000 train acc: 0.7213
Epoch: 1856/2000 test cost: 1.8868
Epoch: 1856/2000 test acc: 0.5724
##################################
Epoch: 1857/2000 train cost: 1.7362
Epoch: 1857/2000 train acc: 0.7308
Epoch: 1857/2000 test cost: 1.8904
Epoch: 1857/2000 test acc: 0.5682
##################################
Epoch: 1858/2000 train cost: 1.7370
Epoch: 1858/2000 train acc: 0.7293
Epoch: 1858/2000 test cost: 1.8874
Epoch: 1858/2000 test acc: 0.5707
##################################
Epoch: 1859/2000 train cost: 1.7412
Epoch: 1859/2000 train acc: 0.7252
Epoch: 1859/2000 te

##################################
Epoch: 1901/2000 train cost: 1.7488
Epoch: 1901/2000 train acc: 0.7173
Epoch: 1901/2000 test cost: 1.8856
Epoch: 1901/2000 test acc: 0.5719
##################################
Epoch: 1902/2000 train cost: 1.7370
Epoch: 1902/2000 train acc: 0.7305
Epoch: 1902/2000 test cost: 1.8866
Epoch: 1902/2000 test acc: 0.5743
##################################
Epoch: 1903/2000 train cost: 1.7420
Epoch: 1903/2000 train acc: 0.7242
Epoch: 1903/2000 test cost: 1.8878
Epoch: 1903/2000 test acc: 0.5713
##################################
Epoch: 1904/2000 train cost: 1.7359
Epoch: 1904/2000 train acc: 0.7320
Epoch: 1904/2000 test cost: 1.8834
Epoch: 1904/2000 test acc: 0.5756
##################################
Epoch: 1905/2000 train cost: 1.7445
Epoch: 1905/2000 train acc: 0.7217
Epoch: 1905/2000 test cost: 1.8829
Epoch: 1905/2000 test acc: 0.5756
##################################
Epoch: 1906/2000 train cost: 1.7303
Epoch: 1906/2000 train acc: 0.7367
Epoch: 1906/2000 te

##################################
Epoch: 1948/2000 train cost: 1.7253
Epoch: 1948/2000 train acc: 0.7405
Epoch: 1948/2000 test cost: 1.8815
Epoch: 1948/2000 test acc: 0.5781
##################################
Epoch: 1949/2000 train cost: 1.7271
Epoch: 1949/2000 train acc: 0.7398
Epoch: 1949/2000 test cost: 1.8891
Epoch: 1949/2000 test acc: 0.5691
##################################
Epoch: 1950/2000 train cost: 1.7419
Epoch: 1950/2000 train acc: 0.7242
Epoch: 1950/2000 test cost: 1.8874
Epoch: 1950/2000 test acc: 0.5711
##################################
Epoch: 1951/2000 train cost: 1.7332
Epoch: 1951/2000 train acc: 0.7327
Epoch: 1951/2000 test cost: 1.8848
Epoch: 1951/2000 test acc: 0.5725
##################################
Epoch: 1952/2000 train cost: 1.7313
Epoch: 1952/2000 train acc: 0.7350
Epoch: 1952/2000 test cost: 1.8877
Epoch: 1952/2000 test acc: 0.5675
##################################
Epoch: 1953/2000 train cost: 1.7253
Epoch: 1953/2000 train acc: 0.7430
Epoch: 1953/2000 te

##################################
Epoch: 1995/2000 train cost: 1.7357
Epoch: 1995/2000 train acc: 0.7293
Epoch: 1995/2000 test cost: 1.8851
Epoch: 1995/2000 test acc: 0.5723
##################################
Epoch: 1996/2000 train cost: 1.7354
Epoch: 1996/2000 train acc: 0.7303
Epoch: 1996/2000 test cost: 1.8834
Epoch: 1996/2000 test acc: 0.5760
##################################
Epoch: 1997/2000 train cost: 1.7335
Epoch: 1997/2000 train acc: 0.7330
Epoch: 1997/2000 test cost: 1.8874
Epoch: 1997/2000 test acc: 0.5692
##################################
Epoch: 1998/2000 train cost: 1.7275
Epoch: 1998/2000 train acc: 0.7387
Epoch: 1998/2000 test cost: 1.8882
Epoch: 1998/2000 test acc: 0.5714
##################################
Epoch: 1999/2000 train cost: 1.7257
Epoch: 1999/2000 train acc: 0.7400
Epoch: 1999/2000 test cost: 1.8847
Epoch: 1999/2000 test acc: 0.5725
